In [1]:
import datetime as dt
import math

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

from fetching_from_local_db.enums import AssetClass, Index, StrikeSpread
from fetching_from_local_db.fetch_from_db import (
    _fetch_batch,
    fetch_data,
    fetch_spot_data,
)

In [2]:
from datetime import date
from bisect import bisect_left

def get_expiry(input_date, index_symbol='nifty'):
    expiries = dict_expiries.get(index_symbol)
    if not expiries:
        return None
        
    expiry_dates = sorted({dt.date() for dt in expiries})
    pos = bisect_left(expiry_dates, input_date)    
    return expiry_dates[pos] if pos < len(expiry_dates) else None


In [3]:
# bnf_pandas = pd.read_csv("../data/bnf_min.csv")
bnf_pandas = pd.read_csv("../data/nifty_min.csv")
# bnf_pandas = pd.read_csv("../data/fin_min.csv")
# bnf_pandas = pd.read_csv("../data/midcp_min.csv")
# bnf_pandas = pd.read_csv("../data/sensex_min.csv")
# bnf_pandas = pd.read_csv("../data/bankex_min.csv")

In [4]:
bnf_pandas.columns = ['index', 'datetime', 'o', 'h', 'l', 'c', 'v']
bnf_pandas.head()

,index,datetime,o,h,l,c,v
0,nifty,2017-01-02 09:15:00,8210.1000,8211.7000,8189.0000,8189.5500,0
1,nifty,2017-01-02 09:16:00,8188.7500,8193.9500,8188.7500,8189.9500,0
2,nifty,2017-01-02 09:17:00,8190.1500,8190.7500,8173.7000,8173.7000,0
3,nifty,2017-01-02 09:18:00,8173.3500,8177.5500,8169.1500,8177.5500,0
4,nifty,2017-01-02 09:19:00,8177.8500,8178.1500,8173.4500,8174.4000,0


In [5]:
# If Stocks Data ...
bnf_pandas["datetime"] = pd.to_datetime(bnf_pandas["datetime"])
bnf_pandas["datetime"] = bnf_pandas["datetime"].dt.tz_localize(None)
bnf_pandas = bnf_pandas[bnf_pandas["datetime"].dt.year >= 2019]
# bnf_pandas.drop(columns=["time"], inplace=True)
# bnf_pandas

In [6]:
bnf = pl.DataFrame(bnf_pandas)
print(type(bnf))
# bnf

<class 'polars.dataframe.frame.DataFrame'>


In [7]:
bnf = bnf.with_columns([pl.col("datetime").alias("index")]).drop("datetime")
bnf = bnf.with_columns(pl.col("index").alias("datetime"))

In [8]:
# bnf = bnf.rename({"open": "o", "high": "h", "low": "l", "close": "c", "volume": "v"})

In [9]:
from expiries import dict_expiries

In [10]:
def resample(data, timeframe, offset=None):
    agg_list = [
        pl.col("o").first().alias("o"),
        pl.col("h").max().alias("h"),
        pl.col("l").min().alias("l"),
        pl.col("c").last().alias("c"),
    ]
    if timeframe == '10m':
        offset = '5m'
    if timeframe == '20m':
        offset = '15m'
    
    if "v" in data.columns:
        agg_list.append(pl.col("v").sum().alias("v"))
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(agg_list)
    )


In [11]:
import pandas as pd

def calculate_rsi(df: pd.DataFrame, period: int = 50) -> pd.Series:
    
    delta = df['c'].diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)

    avg_gain = gain.rolling(window=period, min_periods=period).mean()
    avg_loss = loss.rolling(window=period, min_periods=period).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    df['rsi'] = rsi
    
    return df 


In [73]:
def generate_signals(df, roc_period = 1, period = 6, rsi_threshold = 70):
    
    df['daily_high_till_now'] = df.groupby(df['datetime'].dt.date)['h'].cummax()
    df['ROC'] = ((df['c'] - df['c'].shift(roc_period)) / df['c'].shift(roc_period)) * 100
    roc = df['ROC']
    
    df = calculate_rsi(df, period)
    # df['ma'] = df['c'].rolling(9).mean()
    
    df['Sell Signal'] = (
        (roc < 0) &
        (abs(df['c'] - df['o']) >= 0.6*(df['h'] - df['l']))
    ).astype(int)

    # df['Sell Signal'] = (
    #     (roc < 0) &
    #     (df['rsi'].shift(1) >= rsi_threshold)
    # ).astype(int)
    return df


In [74]:
data = bnf_pandas.copy()
# data['datetime'] = pd.to_datetime(data['datetime'].dt.date)
print(data["datetime"].tail())
trading_days_set = set(data["datetime"].dt.date)
# sorted(trading_days_set)

750760   2025-03-28 15:25:00
750761   2025-03-28 15:26:00
750762   2025-03-28 15:27:00
750763   2025-03-28 15:28:00
750764   2025-03-28 15:29:00
Name: datetime, dtype: datetime64[ns]


In [75]:
# GLOBAL VARIABLES

INSTRUMENT = "NIFTY"
INDEX = "nifty"
INDEX_MROUND = 50
# INDEX_MROUND=100

# INSTRUMENT = "NIFTY"
# INDEX = "nifty"
# INDEX_MROUND=50

PORTFOLIO_VALUE = 1_00_00_000
INDEX_LEV = 7
RPT_CE = 0.01
RPT_PE = 0.01
SLIPPAGE = 0.01
MAX_MARGIN = 100
# TF = "10m"

# SIGNAL_MA = 20
# NUM_OF_CANDELS = 1
# T_MA=20
# TARGET=100

In [76]:
bnf = resample(bnf, '1m')
data = bnf.to_pandas()
data.tail()
# data[['MA','signal_spot']]=MA(data,200)
# data[data['signal_spot']==1].head(50)
# bnf
# data.tail(50)
# data[data['datetime'].dt.date == dt.date(2024, 4, 29)]

,datetime,o,h,l,c,v
19783,2025-03-28 13:00:00,23526.6000,23527.5000,23471.1000,23493.9500,0
19784,2025-03-28 13:30:00,23493.6500,23522.5000,23453.6500,23473.2500,0
19785,2025-03-28 14:00:00,23472.5500,23574.7000,23451.3000,23571.8000,0
19786,2025-03-28 14:30:00,23573.2000,23612.0500,23540.0500,23591.5000,0
19787,2025-03-28 15:00:00,23591.1500,23591.2000,23450.2000,23495.1500,0


In [96]:
async def ce_trade(data, tf, offset, roc_period, spot_rsi_period, rsi_threshold, start_date, end_date):
    df_ = data.copy()
    df_polars = resample(pl.DataFrame(df_), tf, offset)
    df_ = df_polars.to_pandas()
    df = calculate_rsi(df_, spot_rsi_period)
    df['ma'] = df['c'].rolling(9).mean()
    # start_date = dt.date(2024, 1, 5)
    # end_date = dt.date(2025, 3, 31)
    current_date = start_date

    combined_trades = pd.DataFrame()
    total_trades = pd.DataFrame()
    time_of_day = dt.time(9, 15)
    trade_book = []
    ce_lowest_low = float("inf")
    ce_highest_high = float("-inf")
    entry_roc = 0
    opt_rsi = 0
    spot_rsi = 0
    is_trailing_active = False
    trade_num = 0

    while current_date < end_date:
        print(f'CE : {current_date}')
        entry = 0
        initial_sl = 0
        exit = 0
        in_ce_trade = False
        in_pe_trade = False
        # signal_exist=False

        points_captured = 0
        remark = ""
        trailing_active = False
        tsl = 0
        stop_trading = False
        is_gap_ce_sl = False
        previous_ce_sl_hit = False
        current_date_increament_flag = False
        # tsl_high = 0

        starting_time = dt.time(9, 15)

        ending_time = dt.time(15, 30)

        if not in_ce_trade and current_date in trading_days_set:

            ce_search_datetime = dt.datetime.combine(current_date, time_of_day)
            # print(f'current date : {ce_search_datetime}')

            spot_open = df.loc[df["datetime"] >= ce_search_datetime, "o"].iloc[0]
            
            spot_atm = int(
                math.ceil(spot_open / INDEX_MROUND) * INDEX_MROUND
            )  ##ROUNDS TO NEAREST 500 OTM
            
            nearest_expiry = get_expiry(current_date)
            
            selected_strike_ce = spot_atm
            # print(f'selected strike CE : {selected_strike_ce}')
            ce_df = await fetch_data(
                index=INDEX,
                start_date=nearest_expiry - dt.timedelta(days=14),
                start_time=starting_time,
                end_date=nearest_expiry,
                end_time=ending_time,
                strike=selected_strike_ce,
                asset_class="C",
                expiry=nearest_expiry,
            )
            # print(ce_df)
            if ce_df is not None and not isinstance(ce_df, str):
                # print('new data fetched CE')
                data_ce = True
                ce_df = ce_df.select(["datetime", "o", "h", "l", "c", "v"])
                ce_df = resample(ce_df, tf, offset)
                ce_df_pandas = ce_df.to_pandas()
                ce_df_pandas = ce_df_pandas[ce_df_pandas['datetime'].dt.time != dt.time(15, 30)]
                ce_df = generate_signals(ce_df_pandas, roc_period)
                # ce_df = calculate_signals(ce_df_pandas)
                # print(ce_df.to_string())
            else:
                data_ce = False
                current_date += dt.timedelta(days=1)
                continue

            if data_ce:
                ce_df['datetime'] = pd.to_datetime(ce_df['datetime'])
                for i in range(0, len(ce_df)):
                    current_candle = ce_df.iloc[i]
                    current_candle_open = ce_df.iloc[i]["o"]
                    current_candle_high = ce_df.iloc[i]["h"]
                    current_candle_low = ce_df.iloc[i]["l"]
                    current_candle_close = ce_df.iloc[i]["c"]

                    previous_candle_low = ce_df.iloc[i - 1]["l"]
                    previous_candle_close = ce_df.iloc[i - 1]["c"]
                    
                    dte = (nearest_expiry - ce_df.iloc[i-1]['datetime'].date()).days
                    expiry = nearest_expiry
                    strike = selected_strike_ce
                    asset_class = "C"
                    # print(ce_df.iloc[i].to_string())

                    signal = (ce_df.iloc[i - 1]["Sell Signal"]) and (dte in [1, 2, 3]) 
                    # signal = (ce_df.iloc[i - 1]["Sell Signal"])
                    # candle_condition = (ce_df.iloc[i-1]['daily_high_till_now'] - previous_candle_close) > 10

                    if ce_df.iloc[i]["datetime"] >= ce_search_datetime:


                        if (
                            not previous_ce_sl_hit
                            and not in_ce_trade
                            and signal
                            # and current_candle_low < previous_candle_low
                            and ce_df.iloc[i]["datetime"].time() > time_of_day
                            and (
                                (nearest_expiry - ce_df.iloc[i]["datetime"].date()).days
                                >= 0
                                and (
                                    nearest_expiry - ce_df.iloc[i]["datetime"].date()
                                ).days
                                < 7
                            )
                            and ce_df.iloc[i]["datetime"].time() < dt.time(15, 15)
                            # and candle_condition
                        ):
                            # print(ce_df.iloc[i-1])
                            # print('####################################################################################################')
                            # print(f'entry found {current_candle_open}')
                            # print(f'entry datetime {ce_df.iloc[i-1]["datetime"]}')
                            # print('####################################################################################################')
                            trade_num += 1
                            entry = current_candle_open
                            entry_date = ce_df.iloc[i-1]["datetime"].date()
                            entry_time = ce_df.iloc[i-1]["datetime"].time()
                            # initial_sl = ce_df.iloc[i - SL_CANDLES_NUM : i]["h"].max()
                            # initial_sl = entry * 1.3
                            initial_sl = ce_df.iloc[i-1]['daily_high_till_now']
                            in_ce_trade = True
                            ce_lowest_low = float("inf")
                            ce_highest_high = float("-inf")
                            # print(f'initial SL : {initial_sl}')
                            entry_roc = ce_df.iloc[i-1]['ROC']
                            opt_rsi = ce_df.iloc[i-1]['rsi']
                            spot_rsi = df.iloc[i-1]['rsi']
                            # qty = RPT_CE * PORTFOLIO_VALUE / (initial_sl - entry)
                            # if (
                            #     (qty * strike) / (INDEX_LEV * PORTFOLIO_VALUE)
                            # ) * 100 > MAX_MARGIN:
                            #     qty = PORTFOLIO_VALUE * INDEX_LEV / strike * (MAX_MARGIN / 100)
                            qty = PORTFOLIO_VALUE * INDEX_LEV / spot_atm
                            # print(f'qty : {qty}')

                        # While in trade, track the highest high and lowest low
                        if in_ce_trade:
                            # Track the highest high
                            ce_highest_high = max(ce_highest_high, current_candle_high)

                            # Track the lowest low
                            ce_lowest_low = min(ce_lowest_low, current_candle_low)

                            # if ce_lowest_low < entry * (100 - decay) / 100:
                            #     is_trailing_active = True

                            # trailing_sl_signal = ce_df['Trailing Signal'].iloc[i]

                        if (
                            in_ce_trade
                            and ce_df.iloc[i]["datetime"].time() == dt.time(9, 15)
                            and current_candle_open > initial_sl
                        ):

                            exit = current_candle_close
                            in_ce_trade = False
                            stop_trading = False
                            previous_ce_sl_hit = True
                            is_gap_ce_sl = False
                            points_captured = entry - exit
                            exit_time = ce_df.iloc[i]["datetime"].time()
                            slippage = SLIPPAGE * (entry + exit)
                            pnl = qty * (points_captured - slippage)
                            remark = "Gap SL hit"
                            weekday_int = entry_date.weekday()
                            weekday_name = [
                                "Monday",
                                "Tuesday",
                                "Wednesday",
                                "Thursday",
                                "Friday",
                                "Saturday",
                                "Sunday",
                            ][weekday_int]
                            trade = {
                                "date": entry_date,
                                "day": weekday_name,
                                "expiry": expiry,
                                "DTE": (nearest_expiry - entry_date).days,
                                "trade_num": trade_num,
                                # 'atm' : atm,
                                # 'scrip' : index ,
                                "strike": strike,
                                "type": asset_class,
                                "Entry Price": entry,
                                "Entry Time": entry_time,
                                "initial sl": initial_sl,
                                # "TSL": tsl_high,
                                # 'OTM Entry' : otm_entry,
                                "Exit Price": exit,
                                "Exit date": ce_df.iloc[i]["datetime"].date(),
                                "Exit Time": exit_time,
                                'ROC on Entry': entry_roc,
                                'Opt RSI on Entry': opt_rsi,
                                'Spot RSI on Entry': spot_rsi,
                                # 'OTM EXIT ' : otm_exit,
                                "Remark": remark,
                                "Points Captured": points_captured,
                                "Slippage": slippage,
                                # 'OTM cost' : otm_exit-otm_entry,
                                "Qty": qty,
                                "PnL": pnl,
                                "ROI%": (pnl / PORTFOLIO_VALUE) * 100,
                                "Trade Year": ce_df.iloc[i]["datetime"].year,
                                "Trade Month": ce_df.iloc[i]["datetime"].month,
                                "Highest High": ce_highest_high,  # Add highest high to trade data
                                "Lowest Low": ce_lowest_low,  # Add lowest low to trade data
                                "Max ROI%": (
                                    (qty * (entry - ce_lowest_low)) / PORTFOLIO_VALUE
                                )
                                * 100,
                                "Margin": (
                                    (qty * strike) / (INDEX_LEV * PORTFOLIO_VALUE)
                                )
                                * 100,
                            }
                            # print(f'{trade}')
                            trade_book.append(trade)
                            # tsl_high = 0
                            points_captured = 0
                            current_date = ce_df.iloc[i]["datetime"].date()
                            current_date_increament_flag = True
                            time_of_day = ce_df.iloc[i]["datetime"].time()
                            # print(f'current date changed to : {current_date} and time to {time_of_day}')
                            is_trailing_active = False
                            break

                        if in_ce_trade and current_candle_high > initial_sl:
                            exit = initial_sl
                            otm_datetime = ce_df.iloc[i]["datetime"]
                            in_ce_trade = False
                            stop_trading = False
                            previous_ce_sl_hit = True
                            is_gap_ce_sl = False
                            points_captured = entry - exit
                            exit_time = ce_df.iloc[i]["datetime"].time()
                            slippage = SLIPPAGE * (entry + exit)
                            pnl = qty * (points_captured - slippage)
                            # pnl=(qty*(points_captured-slippage))-qty*(otm_exit-otm_entry)
                            remark = "SL hit"
                            weekday_int = entry_date.weekday()
                            weekday_name = [
                                "Monday",
                                "Tuesday",
                                "Wednesday",
                                "Thursday",
                                "Friday",
                                "Saturday",
                                "Sunday",
                            ][weekday_int]
                            trade = {
                                "date": entry_date,
                                "day": weekday_name,
                                "expiry": expiry,
                                "DTE": (nearest_expiry - entry_date).days,
                                "trade_num": trade_num,
                                # 'atm' : atm,
                                # 'scrip' : index ,
                                "strike": strike,
                                "type": asset_class,
                                "Entry Price": entry,
                                "Entry Time": entry_time,
                                "initial sl": initial_sl,
                                # "TSL": tsl_high,
                                # 'OTM Entry' : otm_entry,
                                "Exit Price": exit,
                                "Exit date": ce_df.iloc[i]["datetime"].date(),
                                "Exit Time": exit_time,
                                'ROC on Entry': entry_roc,
                                'Opt RSI on Entry': opt_rsi,
                                'Spot RSI on Entry': spot_rsi,
                                # 'OTM EXIT ' : otm_exit,
                                "Remark": remark,
                                "Points Captured": points_captured,
                                "Slippage": slippage,
                                # 'OTM cost' : otm_exit-otm_entry,
                                "Qty": qty,
                                "PnL": pnl,
                                "ROI%": (pnl / PORTFOLIO_VALUE) * 100,
                                "Trade Year": ce_df.iloc[i]["datetime"].year,
                                "Trade Month": ce_df.iloc[i]["datetime"].month,
                                "Highest High": ce_highest_high,  # Add highest high to trade data
                                "Lowest Low": ce_lowest_low,  # Add lowest low to trade data
                                "Max ROI%": (
                                    (qty * (entry - ce_lowest_low)) / PORTFOLIO_VALUE
                                )
                                * 100,
                                "Margin": (
                                    (qty * strike) / (INDEX_LEV * PORTFOLIO_VALUE)
                                )
                                * 100,
                            }
                            # print(f'{trade}')
                            trade_book.append(trade)
                            # tsl_high = 0
                            points_captured = 0
                            current_date = ce_df.iloc[i]["datetime"].date()
                            current_date_increament_flag = True
                            time_of_day = ce_df.iloc[i]["datetime"].time()
                            # print(f'current date changed to : {current_date} and time to {time_of_day}')
                            is_trailing_active = False
                            break

                        if (
                            in_ce_trade
                            and ce_df.iloc[i]["datetime"].date() == nearest_expiry
                            and ce_df.iloc[i]['datetime'].time() >= dt.time(15, 15)
                        ):
                            # print(ce_df.iloc[i])
                            # print(f'EOD exit {current_candle_close}')
                            # print(f'EOD datetime {ce_df.iloc[i]["datetime"]}')
                            exit = current_candle_close
                            otm_datetime = ce_df.iloc[i]["datetime"]
                            in_ce_trade = False
                            previous_ce_sl_hit = True
                            is_gap_ce_sl = False
                            points_captured = entry - exit
                            exit_time = ce_df.iloc[i]["datetime"].time()
                            slippage = SLIPPAGE * (entry + exit)
                            pnl = qty * (points_captured - slippage)
                            # pnl=(qty*(points_captured-slippage))-qty*(otm_exit-otm_entry)
                            remark = "EOD exit"
                            weekday_int = entry_date.weekday()
                            weekday_name = [
                                "Monday",
                                "Tuesday",
                                "Wednesday",
                                "Thursday",
                                "Friday",
                                "Saturday",
                                "Sunday",
                            ][weekday_int]
                            trade = {
                                "date": entry_date,
                                "day": weekday_name,
                                "expiry": expiry,
                                "DTE": (nearest_expiry - entry_date).days,
                                "trade_num": trade_num,
                                # 'atm' : atm,
                                # 'scrip' : index ,
                                "strike": strike,
                                "type": asset_class,
                                "Entry Price": entry,
                                "Entry Time": entry_time,
                                "initial sl": initial_sl,
                                # "TSL": tsl_high,
                                # 'OTM Entry' : otm_entry,
                                "Exit Price": exit,
                                "Exit date": ce_df.iloc[i]["datetime"].date(),
                                "Exit Time": exit_time,
                                'ROC on Entry': entry_roc,
                                'Opt RSI on Entry': opt_rsi,
                                'Spot RSI on Entry': spot_rsi,
                                # 'OTM EXIT ' : otm_exit,
                                "Remark": remark,
                                "Points Captured": points_captured,
                                "Slippage": slippage,
                                # 'OTM cost' : otm_exit-otm_entry,
                                "Qty": qty,
                                "PnL": pnl,
                                "ROI%": (pnl / PORTFOLIO_VALUE) * 100,
                                "Trade Year": ce_df.iloc[i]["datetime"].year,
                                "Trade Month": ce_df.iloc[i]["datetime"].month,
                                "Highest High": ce_highest_high,  # Add highest high to trade data
                                "Lowest Low": ce_lowest_low,  # Add lowest low to trade data
                                "Max ROI%": (
                                    (qty * (entry - ce_lowest_low)) / PORTFOLIO_VALUE
                                )
                                * 100,
                                "Margin": (
                                    (qty * strike) / (INDEX_LEV * PORTFOLIO_VALUE)
                                )
                                * 100,
                            }
                            # print(f'{trade}')
                            trade_book.append(trade)
                            # tsl_high = 0
                            points_captured = 0
                            current_date = nearest_expiry + dt.timedelta(days=1)
                            current_date_increament_flag = True
                            time_of_day = dt.time(9, 15)
                            # print(f'current date increased by 1 on expiry : {current_date}')
                            is_trailing_active = False
                            trade_num = 0
                            break

                        if (
                            not in_ce_trade
                            and not previous_ce_sl_hit
                            and ce_df.iloc[i]["datetime"].time() >= dt.time(15, 15)
                        ):
                            # print('inside exoiry non trade date increment')
                            current_date = current_date + dt.timedelta(days=1)
                            current_date_increament_flag = True
                            time_of_day = dt.time(9, 15)
                            break

        if not current_date_increament_flag:
            current_date = current_date + dt.timedelta(days=1)
            current_date_increament_flag = False

    trade_book_df = pd.DataFrame(trade_book)

    return trade_book_df

In [101]:
async def pe_trade(data, tf, offset, roc_period, spot_rsi_period, rsi_threshold, start_date, end_date):
    df_ = data.copy()
    df_polars = resample(pl.DataFrame(df_), tf, offset)
    df_ = df_polars.to_pandas()
    df = calculate_rsi(df_, spot_rsi_period)
    df['ma'] = df['c'].rolling(9).mean()
    # start_date = dt.date(2024, 1, 5)
    # end_date = dt.date(2025, 3, 31)
    current_date = start_date

    combined_trades = pd.DataFrame()
    total_trades = pd.DataFrame()
    time_of_day = dt.time(9, 15)
    trade_book = []
    pe_lowest_low = float("inf")
    pe_highest_high = float("-inf")
    entry_roc = 0
    opt_rsi = 0
    spot_rsi = 0
    is_trailing_active = False
    trade_num = 0

    while current_date < end_date:
        print(f'PE : {current_date}')
        entry = 0
        initial_sl = 0
        exit = 0
        in_pe_trade = False
        in_pe_trade = False
        # signal_exist=False

        points_captured = 0
        remark = ""
        trailing_active = False
        tsl = 0
        stop_trading = False
        is_gap_pe_sl = False
        previous_pe_sl_hit = False
        current_date_increament_flag = False
        # tsl_high = 0

        starting_time = dt.time(9, 15)

        ending_time = dt.time(15, 30)

        if not in_pe_trade and current_date in trading_days_set:

            pe_search_datetime = dt.datetime.combine(current_date, time_of_day)
            # print(f'current date : {pe_search_datetime}')

            spot_open = df.loc[df["datetime"] >= pe_search_datetime, "o"].iloc[0]
            # print(f'spot open : {spot_open}')
            # spot_atm = int(round(spot_open / INDEX_MROUND) * INDEX_MROUND)
            spot_atm = int(
                math.floor(spot_open / INDEX_MROUND) * INDEX_MROUND
            )  ##ROUNDS TO NEAREST 500 OTM
            # print(f'spot atm : {spot_atm}')
            # nearest_expiry = await get_expiry(current_date)
            nearest_expiry = get_expiry(current_date)
            # if current_date== nearest_expiry:
            #     next_expiry_passing_value = current_date + dt.timedelta(days=1)
            #     nearest_expiry = await get_expiry_nifty( next_expiry_passing_value)
            # print(f'passing date for expry : {current_date}')
            # nearest_expiry = await get_monthly_expiry_nifty(current_date)
            # print(f'nearest expiry{nearest_expiry}')
            selected_strike_pe = spot_atm
            # print(f'selected strike PE : {selected_strike_pe}')
            pe_df = await fetch_data(
                index=INDEX,
                start_date=nearest_expiry - dt.timedelta(days=14),
                start_time=starting_time,
                end_date=nearest_expiry,
                end_time=ending_time,
                strike=selected_strike_pe,
                asset_class="P",
                expiry=nearest_expiry,
            )
            if pe_df is not None and not isinstance(pe_df, str):
                # print('new data fetched PE')
                data_pe = True
                pe_df = pe_df.select(["datetime", "o", "h", "l", "c", "v"])
                pe_df = resample(pe_df, tf, offset)
                pe_df_pandas = pe_df.to_pandas()
                pe_df_pandas = pe_df_pandas[pe_df_pandas['datetime'].dt.time != dt.time(15, 30)]
                pe_df = generate_signals(pe_df_pandas, roc_period)
                # pe_df = calculate_signals(pe_df_pandas)
                # print(pe_df.to_string())
            else:
                data_pe = False
                current_date += dt.timedelta(days=1)
                continue

            if data_pe:
                pe_df['datetime'] = pd.to_datetime(pe_df['datetime'])
                for i in range(0, len(pe_df)):
                    current_candle = pe_df.iloc[i]
                    current_candle_open = pe_df.iloc[i]["o"]
                    current_candle_high = pe_df.iloc[i]["h"]
                    current_candle_low = pe_df.iloc[i]["l"]
                    current_candle_close = pe_df.iloc[i]["c"]

                    previous_candle_low = pe_df.iloc[i - 1]["l"]
                    previous_candle_close = pe_df.iloc[i - 1]["c"]
                    
                    dte = (nearest_expiry - pe_df.iloc[i-1]['datetime'].date()).days
                    expiry = nearest_expiry
                    strike = selected_strike_pe
                    asset_class = "P"
                    # print(pe_df.iloc[i].to_string())

                    signal = (pe_df.iloc[i - 1]["Sell Signal"]) and (dte in [1, 2, 3])
                    # signal = (pe_df.iloc[i - 1]["Sell Signal"])
                    # candle_condition = (pe_df.iloc[i-1]['daily_high_till_now'] - previous_candle_close) > 10

                    if pe_df.iloc[i]["datetime"] >= pe_search_datetime:


                        if (
                            not previous_pe_sl_hit
                            and not in_pe_trade
                            and signal
                            # and current_candle_low < previous_candle_low
                            and pe_df.iloc[i]["datetime"].time() > time_of_day
                            and (
                                (nearest_expiry - pe_df.iloc[i]["datetime"].date()).days
                                >= 0
                                and (
                                    nearest_expiry - pe_df.iloc[i]["datetime"].date()
                                ).days
                                < 7
                            )
                            and pe_df.iloc[i]["datetime"].time() < dt.time(15, 15)
                            # and candle_condition
                        ):
                            # print(pe_df.iloc[i-1])
                            # print('####################################################################################################')
                            # print(f'entry found {current_candle_open}')
                            # print(f'entry datetime {pe_df.iloc[i-1]["datetime"]}')
                            # print('####################################################################################################')
                            
                            # today_data = pe_df[pe_df['datetime'].dt.date == current_candle['datetime'].date()]
                            # day_high = today_data.iloc[0 : i]['h'].max()
                            # print(today_data.to_string())
                            trade_num += 1
                            entry = current_candle_open
                            entry_date = pe_df.iloc[i-1]["datetime"].date()
                            entry_time = pe_df.iloc[i-1]["datetime"].time()
                            # initial_sl = pe_df.iloc[i - SL_CANDLES_NUM : i]["h"].max()
                            # initial_sl = entry * 1.3
                            initial_sl = pe_df.iloc[i-1]['daily_high_till_now']
                            in_pe_trade = True
                            pe_lowest_low = float("inf")
                            pe_highest_high = float("-inf")
                            entry_roc = pe_df.iloc[i-1]['ROC']
                            opt_rsi = pe_df.iloc[i-1]['rsi']
                            spot_rsi = df.iloc[i-1]['rsi']
                            # qty = RPT_PE * PORTFOLIO_VALUE / (initial_sl - entry)
                            # if (
                            #     (qty * strike) / (INDEX_LEV * PORTFOLIO_VALUE)
                            # ) * 100 > MAX_MARGIN:
                            #     qty = PORTFOLIO_VALUE * INDEX_LEV / strike * (MAX_MARGIN / 100)

                            qty = PORTFOLIO_VALUE * INDEX_LEV / spot_atm

                        # While in trade, track the highest high and lowest low
                        if in_pe_trade:
                            # Track the highest high
                            pe_highest_high = max(pe_highest_high, current_candle_high)

                            # Track the lowest low
                            pe_lowest_low = min(pe_lowest_low, current_candle_low)
                            
                            # if pe_lowest_low < entry * (100 - decay) / 100:
                            #     is_trailing_active = True

                            # trailing_sl_signal = pe_df['Trailing Signal'].iloc[i]
                           
                        if (
                            in_pe_trade
                            and pe_df.iloc[i]["datetime"].time() == dt.time(9, 15)
                            and current_candle_open > initial_sl
                        ):

                            # print(pe_df.iloc[i])
                            # print(f'GAP sl hit {initial_sl}')
                            # print(f'GAP sl datetime {pe_df.iloc[i]["datetime"]}')
                            exit = current_candle_close
                            in_pe_trade = False
                            stop_trading = False
                            previous_pe_sl_hit = True
                            is_gap_pe_sl = False
                            points_captured = entry - exit
                            exit_time = pe_df.iloc[i]["datetime"].time()
                            slippage = SLIPPAGE * (entry + exit)
                            pnl = qty * (points_captured - slippage)
                            remark = "Gap SL hit"
                            weekday_int = entry_date.weekday()
                            weekday_name = [
                                "Monday",
                                "Tuesday",
                                "Wednesday",
                                "Thursday",
                                "Friday",
                                "Saturday",
                                "Sunday",
                            ][weekday_int]
                            trade = {
                                "date": entry_date,
                                "day": weekday_name,
                                "expiry": expiry,
                                "DTE": (nearest_expiry - entry_date).days,
                                "trade_num": trade_num,
                                # 'atm' : atm,
                                # 'scrip' : index ,
                                "strike": strike,
                                "type": asset_class,
                                "Entry Price": entry,
                                "Entry Time": entry_time,
                                "initial sl": initial_sl,
                                # "TSL": tsl_high,
                                # 'OTM Entry' : otm_entry,
                                "Exit Price": exit,
                                "Exit date": pe_df.iloc[i]["datetime"].date(),
                                "Exit Time": exit_time,
                                'ROC on Entry': entry_roc,
                                'Opt RSI on Entry': opt_rsi,
                                'Spot RSI on Entry': spot_rsi,
                                # 'OTM EXIT ' : otm_exit,
                                "Remark": remark,
                                "Points Captured": points_captured,
                                "Slippage": slippage,
                                # 'OTM cost' : otm_exit-otm_entry,
                                "Qty": qty,
                                "PnL": pnl,
                                "ROI%": (pnl / PORTFOLIO_VALUE) * 100,
                                "Trade Year": pe_df.iloc[i]["datetime"].year,
                                "Trade Month": pe_df.iloc[i]["datetime"].month,
                                "Highest High": pe_highest_high,  # Add highest high to trade data
                                "Lowest Low": pe_lowest_low,  # Add lowest low to trade data
                                "Max ROI%": (
                                    (qty * (entry - pe_lowest_low)) / PORTFOLIO_VALUE
                                )
                                * 100,
                                "Margin": (
                                    (qty * strike) / (INDEX_LEV * PORTFOLIO_VALUE)
                                )
                                * 100,
                            }
                            # print(f'{trade}')
                            trade_book.append(trade)
                            # tsl_high = 0
                            points_captured = 0
                            current_date = pe_df.iloc[i]["datetime"].date()
                            current_date_increament_flag = True
                            time_of_day = pe_df.iloc[i]["datetime"].time()
                            # print(f'current date changed to : {current_date} and time to {time_of_day}')
                            is_trailing_active = False
                            break

                        if in_pe_trade and current_candle_high > initial_sl:
                            # print(pe_df.iloc[i])
                            # print(f'initial sl hit {initial_sl}')
                            # print(f'initial sl datetime {pe_df.iloc[i]["datetime"]}')
                            exit = initial_sl
                            otm_datetime = pe_df.iloc[i]["datetime"]
                            in_pe_trade = False
                            stop_trading = False
                            previous_pe_sl_hit = True
                            is_gap_pe_sl = False
                            points_captured = entry - exit
                            exit_time = pe_df.iloc[i]["datetime"].time()
                            slippage = SLIPPAGE * (entry + exit)
                            pnl = qty * (points_captured - slippage)
                            # pnl=(qty*(points_captured-slippage))-qty*(otm_exit-otm_entry)
                            remark = "SL hit"
                            weekday_int = entry_date.weekday()
                            weekday_name = [
                                "Monday",
                                "Tuesday",
                                "Wednesday",
                                "Thursday",
                                "Friday",
                                "Saturday",
                                "Sunday",
                            ][weekday_int]
                            trade = {
                                "date": entry_date,
                                "day": weekday_name,
                                "expiry": expiry,
                                "DTE": (nearest_expiry - entry_date).days,
                                "trade_num": trade_num,
                                # 'atm' : atm,
                                # 'scrip' : index ,
                                "strike": strike,
                                "type": asset_class,
                                "Entry Price": entry,
                                "Entry Time": entry_time,
                                "initial sl": initial_sl,
                                # "TSL": tsl_high,
                                # 'OTM Entry' : otm_entry,
                                "Exit Price": exit,
                                "Exit date": pe_df.iloc[i]["datetime"].date(),
                                "Exit Time": exit_time,
                                'ROC on Entry': entry_roc,
                                'Opt RSI on Entry': opt_rsi,
                                'Spot RSI on Entry': spot_rsi,
                                # 'OTM EXIT ' : otm_exit,
                                "Remark": remark,
                                "Points Captured": points_captured,
                                "Slippage": slippage,
                                # 'OTM cost' : otm_exit-otm_entry,
                                "Qty": qty,
                                "PnL": pnl,
                                "ROI%": (pnl / PORTFOLIO_VALUE) * 100,
                                "Trade Year": pe_df.iloc[i]["datetime"].year,
                                "Trade Month": pe_df.iloc[i]["datetime"].month,
                                "Highest High": pe_highest_high,  # Add highest high to trade data
                                "Lowest Low": pe_lowest_low,  # Add lowest low to trade data
                                "Max ROI%": (
                                    (qty * (entry - pe_lowest_low)) / PORTFOLIO_VALUE
                                )
                                * 100,
                                "Margin": (
                                    (qty * strike) / (INDEX_LEV * PORTFOLIO_VALUE)
                                )
                                * 100,
                            }
                            # print(f'{trade}')
                            trade_book.append(trade)
                            tsl_high = 0
                            points_captured = 0
                            current_date = pe_df.iloc[i]["datetime"].date()
                            current_date_increament_flag = True
                            time_of_day = pe_df.iloc[i]["datetime"].time()
                            # print(f'current date changed to : {current_date} and time to {time_of_day}')
                            is_trailing_active = False
                            break

                        if (
                            in_pe_trade
                            and pe_df.iloc[i]["datetime"].date() == nearest_expiry
                            and pe_df.iloc[i]['datetime'].time() >= dt.time(15, 15)
                        ):
                            # print(pe_df.iloc[i])
                            # print(f'EOD exit {current_candle_close}')
                            # print(f'EOD datetime {pe_df.iloc[i]["datetime"]}')
                            exit = current_candle_close
                            otm_datetime = pe_df.iloc[i]["datetime"]
                            in_pe_trade = False
                            previous_pe_sl_hit = True
                            is_gap_pe_sl = False
                            points_captured = entry - exit
                            exit_time = pe_df.iloc[i]["datetime"].time()
                            slippage = SLIPPAGE * (entry + exit)
                            pnl = qty * (points_captured - slippage)
                            # pnl=(qty*(points_captured-slippage))-qty*(otm_exit-otm_entry)
                            remark = "EOD exit"
                            weekday_int = entry_date.weekday()
                            weekday_name = [
                                "Monday",
                                "Tuesday",
                                "Wednesday",
                                "Thursday",
                                "Friday",
                                "Saturday",
                                "Sunday",
                            ][weekday_int]
                            trade = {
                                "date": entry_date,
                                "day": weekday_name,
                                "expiry": expiry,
                                "DTE": (nearest_expiry - entry_date).days,
                                "trade_num": trade_num,
                                # 'atm' : atm,
                                # 'scrip' : index ,
                                "strike": strike,
                                "type": asset_class,
                                "Entry Price": entry,
                                "Entry Time": entry_time,
                                "initial sl": initial_sl,
                                # "TSL": tsl_high,
                                # 'OTM Entry' : otm_entry,
                                "Exit Price": exit,
                                "Exit date": pe_df.iloc[i]["datetime"].date(),
                                "Exit Time": exit_time,
                                'ROC on Entry': entry_roc,
                                'Opt RSI on Entry': opt_rsi,
                                'Spot RSI on Entry': spot_rsi,
                                # 'OTM EXIT ' : otm_exit,
                                "Remark": remark,
                                "Points Captured": points_captured,
                                "Slippage": slippage,
                                # 'OTM cost' : otm_exit-otm_entry,
                                "Qty": qty,
                                "PnL": pnl,
                                "ROI%": (pnl / PORTFOLIO_VALUE) * 100,
                                "Trade Year": pe_df.iloc[i]["datetime"].year,
                                "Trade Month": pe_df.iloc[i]["datetime"].month,
                                "Highest High": pe_highest_high,  # Add highest high to trade data
                                "Lowest Low": pe_lowest_low,  # Add lowest low to trade data
                                "Max ROI%": (
                                    (qty * (entry - pe_lowest_low)) / PORTFOLIO_VALUE
                                )
                                * 100,
                                "Margin": (
                                    (qty * strike) / (INDEX_LEV * PORTFOLIO_VALUE)
                                )
                                * 100,
                            }
                            # print(f'{trade}')
                            trade_book.append(trade)
                            # tsl_high = 0
                            points_captured = 0
                            current_date = nearest_expiry + dt.timedelta(days=1)
                            current_date_increament_flag = True
                            time_of_day = dt.time(9, 15)
                            # print(f'current date increased by 1 on expiry : {current_date}')
                            is_trailing_active = False
                            trade_num = 0
                            break

                        if (
                            not in_pe_trade
                            and not previous_pe_sl_hit
                            and pe_df.iloc[i]["datetime"].time() >= dt.time(15, 15)
                        ):
                            # print('inside exoiry non trade date increment')
                            current_date = current_date + dt.timedelta(days=1)
                            current_date_increament_flag = True
                            time_of_day = dt.time(9, 15)
                            break

        if not current_date_increament_flag:
            current_date = current_date + dt.timedelta(days=1)
            current_date_increament_flag = False

    trade_book_df = pd.DataFrame(trade_book)

    return trade_book_df

In [102]:
# tb_ce = pd.DataFrame()
# tb_pe = pd.DataFrame()

async def execute(DF, tf, offset, roc_period, spot_rsi_period, rsi_threshold, start_dt, end_dt):
    data = DF.copy()
    tb_ce = await ce_trade(data, tf, offset, roc_period, spot_rsi_period, rsi_threshold, start_dt, end_dt)
    tb_pe = await pe_trade(data, tf, offset, roc_period, spot_rsi_period, rsi_threshold, start_dt, end_dt)
    tb = pd.concat([tb_ce, tb_pe], ignore_index=True)
    # print(len(tb))
    if len(tb)>0:
        tb = tb.sort_values(by="date")
    return tb

In [103]:
def generate_stats(tb_expiry, ema_window):
    stats_df8 = pd.DataFrame(
        index=range(2019, 2026),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe

    # Iterate over each year
    for year in range(2019, 2026):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]

        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()

        # Calculate total number of trades
        total_trades = len(year_trades)

        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100

        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()

        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()

        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()

        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = f"{ema_window}"

        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]

    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0][
        "ROI%"
    ].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum()
        - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation

    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    return {overall_roi_dd_ratio: stats_df8}

In [104]:
tf1 = '30m'
offset1 = '15m'
roc_period_ = 25
spot_rsi_period_ = 9
rsi_threshold_ = 70

start_date = dt.date(2019, 1, 1)
end_date = dt.date(2025, 3, 31)

bnf = resample(bnf, tf1)
data = bnf.to_pandas()
tb = await execute(data, tf1, offset1, roc_period_, spot_rsi_period_, rsi_threshold_, start_date, end_date)

CE : 2019-01-01
CE : 2019-01-02
CE : 2019-01-03
CE : 2019-01-04
CE : 2019-01-05
CE : 2019-01-06
CE : 2019-01-07
CE : 2019-01-08
CE : 2019-01-09
CE : 2019-01-10
CE : 2019-01-11
CE : 2019-01-12
CE : 2019-01-13
CE : 2019-01-14
CE : 2019-01-15
CE : 2019-01-16
CE : 2019-01-17
CE : 2019-01-18
CE : 2019-01-19
CE : 2019-01-20
CE : 2019-01-21
CE : 2019-01-22
CE : 2019-01-23
CE : 2019-01-24
CE : 2019-01-25
CE : 2019-01-26
CE : 2019-01-27
CE : 2019-01-28
CE : 2019-02-01
CE : 2019-02-02
CE : 2019-02-03
CE : 2019-02-04
CE : 2019-02-05
CE : 2019-02-06
CE : 2019-02-07
CE : 2019-02-08
CE : 2019-02-09
CE : 2019-02-10
CE : 2019-02-11
CE : 2019-02-12
CE : 2019-02-13
CE : 2019-02-15
CE : 2019-02-16
CE : 2019-02-17
CE : 2019-02-18
CE : 2019-02-22
CE : 2019-02-23
CE : 2019-02-24
CE : 2019-02-25
CE : 2019-02-25
CE : 2019-02-26
CE : 2019-02-26
CE : 2019-02-27
CE : 2019-03-01
CE : 2019-03-02
CE : 2019-03-03
CE : 2019-03-04
CE : 2019-03-05
CE : 2019-03-05
CE : 2019-03-05
CE : 2019-03-06
CE : 2019-03-07
CE : 201

In [105]:
tb['DATETIME'] = pd.to_datetime(tb['date'].astype(str) + ' ' + tb['Entry Time'].astype(str))

In [106]:
tb = tb.sort_values(by='DATETIME')
stats = generate_stats(tb, 'Kitkat Base')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,40.5540,146,37.6712,2.3667,-0.9848,-15.7662,2.5722,Kitkat Base
2020,174.6507,150,46.0000,4.8119,-1.9429,-23.1566,7.5422,Kitkat Base
2021,35.2248,171,38.0117,2.7014,-1.3242,-13.1274,2.6833,Kitkat Base
2022,35.2345,182,36.2637,3.1964,-1.5149,-25.5641,1.3783,Kitkat Base
2023,3.2204,167,35.3293,1.8153,-0.9619,-11.6146,0.2773,Kitkat Base
2024,2.0838,179,34.0782,2.3758,-1.2105,-18.8267,0.1107,Kitkat Base
2025,27.8609,39,51.2821,2.5873,-1.2571,-4.4731,6.2286,Kitkat Base
Overall,318.8292,1034,38.2012,2.9178,-1.3047,-25.5641,12.4717,Kitkat Base


In [40]:
tb = tb.sort_values(by='DATETIME')
stats = generate_stats(tb, 'Kitkat Base')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,-2.4374,93,40.8602,1.5939,-1.1455,-16.6882,-0.1461,Kitkat Base
2020,3.7406,115,44.3478,2.6549,-2.0572,-20.9996,0.1781,Kitkat Base
2021,14.1169,126,50.0000,2.0048,-1.7808,-25.9280,0.5445,Kitkat Base
2022,31.8930,128,45.3125,2.5131,-1.6267,-21.5531,1.4797,Kitkat Base
2023,-4.1558,132,44.6970,1.4324,-1.2146,-26.2733,-0.1582,Kitkat Base
2024,-21.7165,124,44.3548,1.4179,-1.4450,-24.2989,-0.8937,Kitkat Base
2025,3.7952,33,54.5455,1.8339,-1.9476,-5.9767,0.6350,Kitkat Base
Overall,25.2360,751,45.5393,1.9402,-1.5607,-52.0909,0.4845,Kitkat Base


In [23]:
tb.tail()

,date,day,expiry,DTE,trade_num,strike,type,Entry Price,Entry Time,initial sl,Exit Price,Exit date,Exit Time,ROC on Entry,Opt RSI on Entry,Spot RSI on Entry,Remark,Points Captured,Slippage,Qty,PnL,ROI%,Trade Year,Trade Month,Highest High,Lowest Low,Max ROI%,Margin,DATETIME
696,2025-03-17,Monday,2025-03-20,3,1,22550,C,104.0000,09:45:00,161.4500,161.4500,2025-03-18,09:15:00,-18.3529,57.8862,75.8992,SL hit,-57.4500,2.6545,3104.2129,-186577.1619,-1.8658,2025,3,232.6000,75.8500,0.8738,100.0000,2025-03-17 09:45:00
1318,2025-03-21,Friday,2025-03-27,6,1,23150,P,76.7000,09:15:00,130.0000,0.1000,2025-03-27,15:15:00,-74.5335,12.7483,55.0293,EOD exit,76.6000,0.7680,3023.7581,229297.6242,2.2930,2025,3,86.8500,0.0500,2.3177,100.0000,2025-03-21 09:15:00
697,2025-03-26,Wednesday,2025-03-27,1,2,23700,C,59.0500,11:45:00,132.5000,0.0500,2025-03-27,15:15:00,-47.9965,26.4051,31.3644,EOD exit,59.0000,0.5910,2953.5865,172516.0338,1.7252,2025,3,73.1500,0.0500,1.7426,100.0000,2025-03-26 11:45:00
1319,2025-03-28,Friday,2025-04-03,6,1,23550,P,101.1000,10:15:00,163.7000,163.7000,2025-03-28,13:45:00,-7.1494,45.8546,62.0900,SL hit,-62.6000,2.6480,2972.3992,-193943.0998,-1.9394,2025,3,177.6000,89.1500,0.3552,100.0000,2025-03-28 10:15:00
1320,2025-03-28,Friday,2025-04-03,6,2,23450,P,69.5000,14:15:00,123.8500,123.8500,2025-04-01,12:45:00,-40.2934,48.4144,64.5735,SL hit,-54.3500,1.9335,2985.0746,-168010.4478,-1.6801,2025,4,263.0000,67.4000,0.0627,100.0000,2025-03-28 14:15:00


In [25]:
tb.tail(25)

,date,day,expiry,DTE,trade_num,strike,type,Entry Price,Entry Time,initial sl,Exit Price,Exit date,Exit Time,ROC on Entry,Opt RSI on Entry,Spot RSI on Entry,Remark,Points Captured,Slippage,Qty,PnL,ROI%,Trade Year,Trade Month,Highest High,Lowest Low,Max ROI%,Margin,DATETIME
688,2025-01-24,Friday,2025-01-30,6,2,23300,C,92.1000,14:15:00,215.5000,0.1500,2025-01-30,15:15:00,-38.5971,0.0000,52.1919,EOD exit,91.9500,0.9225,3004.2918,273473.1760,2.7347,2025,1,107.1000,0.1000,2.7639,100.0000,2025-01-24 14:15:00
1306,2025-01-28,Tuesday,2025-01-30,2,2,22900,P,73.8000,13:45:00,182.0000,0.1000,2025-01-30,15:15:00,-22.8033,17.6211,17.8947,EOD exit,73.7000,0.7390,3056.7686,223024.8908,2.2302,2025,1,129.3500,0.0500,2.2544,100.0000,2025-01-28 13:45:00
1307,2025-01-31,Friday,2025-02-06,6,1,23300,P,211.0000,10:15:00,305.1000,0.0500,2025-02-06,15:15:00,-51.1851,0.0000,74.1067,EOD exit,210.9500,2.1105,3004.2918,627414.8069,6.2741,2025,2,222.6000,0.0500,6.3376,100.0000,2025-01-31 10:15:00
689,2025-02-01,Saturday,2025-02-06,5,1,23550,C,169.7500,11:45:00,295.1500,53.2000,2025-02-06,15:15:00,-30.0640,18.9699,43.9187,EOD exit,116.5500,2.2295,2972.3992,339806.1571,3.3981,2025,2,233.7500,14.0000,4.6295,100.0000,2025-02-01 11:45:00
690,2025-02-07,Friday,2025-02-13,6,1,23600,C,194.4500,10:45:00,234.8500,234.8500,2025-02-07,11:15:00,-33.0164,39.9527,50.6994,SL hit,-40.4000,4.2930,2966.1017,-132563.9831,-1.3256,2025,2,248.8000,192.0500,0.0712,100.0000,2025-02-07 10:45:00
1308,2025-02-07,Friday,2025-02-13,6,1,23550,P,91.0000,11:15:00,177.9000,177.9000,2025-02-07,13:45:00,-22.9764,20.6309,70.6187,SL hit,-86.9000,2.6890,2972.3992,-266294.2675,-2.6629,2025,2,200.0000,88.5500,0.0728,100.0000,2025-02-07 11:15:00
691,2025-02-07,Friday,2025-02-13,6,2,23700,C,108.6500,11:45:00,186.8000,0.0500,2025-02-13,15:15:00,-51.9876,32.5479,43.9187,EOD exit,108.6000,1.0870,2953.5865,317548.9451,3.1755,2025,2,123.3000,0.0500,3.2076,100.0000,2025-02-07 11:45:00
1309,2025-02-13,Thursday,2025-02-13,0,2,23000,P,20.4500,09:45:00,105.0000,0.0500,2025-02-13,15:15:00,-24.0672,0.0000,88.2979,EOD exit,20.4000,0.2050,3043.4783,61463.0435,0.6146,2025,2,31.4000,0.0500,0.6209,100.0000,2025-02-13 09:45:00
1310,2025-02-14,Friday,2025-02-20,6,1,23000,P,202.5500,10:15:00,209.0000,209.0000,2025-02-14,10:45:00,-7.9183,71.7878,74.1067,SL hit,-6.4500,4.1155,3043.4783,-32155.8696,-0.3216,2025,2,248.2500,192.8500,0.2952,100.0000,2025-02-14 10:15:00
692,2025-02-14,Friday,2025-02-20,6,1,23050,C,136.9500,10:15:00,236.0000,0.0500,2025-02-20,15:15:00,-21.4796,13.8153,78.5698,EOD exit,136.9000,1.3700,3036.8764,411587.8525,4.1159,2025,2,158.1000,0.0500,4.1575,100.0000,2025-02-14 10:15:00


In [26]:
tb_eod = tb[tb['Remark'] == 'EOD exit']
tb_eod['Entry Price'].mean()
# tb['Points Captured'].min()

67.61379310344827

In [27]:
tb = tb.sort_values(by='DATETIME')
stats = generate_stats(tb, '...')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,40.6219,200,39.5000,2.2096,-1.1069,-24.9886,1.6256,...
2020,210.8000,204,45.0980,4.6174,-1.9107,-34.4821,6.1133,...
2021,30.4095,228,38.5965,2.6788,-1.4666,-37.6326,0.8081,...
2022,51.0958,224,39.7321,3.1180,-1.6771,-38.2511,1.3358,...
2023,38.7879,211,39.8104,1.8722,-0.9329,-9.4427,4.1077,...
2024,-2.6905,210,40.0000,2.1008,-1.4219,-29.1357,-0.0923,...
2025,22.0462,44,52.2727,2.6580,-1.8613,-8.1611,2.7014,...
Overall,391.0708,1321,40.8024,2.7968,-1.4276,-38.2511,10.2238,...


# Moving Parts : 
## ROC period
## TF
## No. of entries per week
## RSI (optional)
## MA for Spot / Options - Both
## SL -> Day high, Prev Candle High, x% from entry, etc.
## Targets

In [28]:
# tb.to_csv('Hammer_10m_15_50.csv', index=False)

In [84]:
# tb = tb_with_hedge
tb['Cumulative ROI%'] = tb['ROI%'].cumsum()
tb['Max Cumulative ROI%'] = tb['Cumulative ROI%'].cummax()  # Maximum value so far
tb['DD'] = tb['Cumulative ROI%'] - tb['Max Cumulative ROI%']  # Drawdown
tb.tail()

,date,day,expiry,DTE,trade_num,strike,type,Entry Price,Entry Time,initial sl,Exit Price,Exit date,Exit Time,ROC on Entry,Opt RSI on Entry,Spot RSI on Entry,Remark,Points Captured,Slippage,Qty,PnL,ROI%,Trade Year,Trade Month,Highest High,Lowest Low,Max ROI%,Margin,DATETIME,Cumulative ROI%,Max Cumulative ROI%,DD
696,2025-03-17,Monday,2025-03-20,3,1,22550,C,104.0000,09:45:00,161.4500,161.4500,2025-03-18,09:15:00,-18.3529,55.8212,83.7805,SL hit,-57.4500,2.6545,3104.2129,-186577.1619,-1.8658,2025,3,232.6000,75.8500,0.8738,100.0000,2025-03-17 09:45:00,390.6722,392.5379,-1.8658
1318,2025-03-21,Friday,2025-03-27,6,1,23150,P,76.7000,09:15:00,130.0000,0.1000,2025-03-27,15:15:00,-74.5335,10.8451,76.8507,EOD exit,76.6000,0.7680,3023.7581,229297.6242,2.2930,2025,3,86.8500,0.0500,2.3177,100.0000,2025-03-21 09:15:00,392.9651,392.9651,0.0000
697,2025-03-26,Wednesday,2025-03-27,1,2,23700,C,59.0500,11:45:00,132.5000,0.0500,2025-03-27,15:15:00,-47.9965,21.1196,29.9660,EOD exit,59.0000,0.5910,2953.5865,172516.0338,1.7252,2025,3,73.1500,0.0500,1.7426,100.0000,2025-03-26 11:45:00,394.6903,394.6903,0.0000
1319,2025-03-28,Friday,2025-04-03,6,1,23550,P,101.1000,10:15:00,163.7000,163.7000,2025-03-28,13:45:00,-7.1494,46.6210,52.2137,SL hit,-62.6000,2.6480,2972.3992,-193943.0998,-1.9394,2025,3,177.6000,89.1500,0.3552,100.0000,2025-03-28 10:15:00,392.7509,394.6903,-1.9394
1320,2025-03-28,Friday,2025-04-03,6,2,23450,P,69.5000,14:15:00,123.8500,123.8500,2025-04-01,12:45:00,-40.2934,42.9912,74.0017,SL hit,-54.3500,1.9335,2985.0746,-168010.4478,-1.6801,2025,4,263.0000,67.4000,0.0627,100.0000,2025-03-28 14:15:00,391.0708,394.6903,-3.6195


In [85]:
tb_ce = tb[tb['type'] == 'C']
tb_pe = tb[tb['type'] == 'P']

In [86]:
import pandas as pd

def calculate_opt_type_stats(df):
    if 'type' not in df.columns or 'ROI%' not in df.columns or 'DD' not in df.columns:
        raise ValueError("DataFrame must contain 'type', 'ROI%', and 'DD' columns.")
    
    # Calculate win flag
    df['is_win'] = df['ROI%'] > 0

    # Group by 'type' and calculate stats
    grouped_stats = df.groupby('type').agg(
        returns_sum=('ROI%', 'sum'),
        max_drawdown=('DD', 'min'),
        total_trades=('type', 'count'),
        wins=('is_win', 'sum')
    ).reset_index()

    # Add calculated columns
    grouped_stats['returns_to_max_dd_ratio'] = grouped_stats['returns_sum'] / grouped_stats['max_drawdown'].abs()
    grouped_stats['win_rate'] = (grouped_stats['wins'] / grouped_stats['total_trades']) * 100

    return grouped_stats

import pandas as pd

def calculate_dte_stats(df):
    if 'DTE' not in df.columns or 'ROI%' not in df.columns or 'DD' not in df.columns:
        raise ValueError("DataFrame must contain 'DTE', 'ROI%', and 'DD' columns.")
    
    df['is_win'] = df['ROI%'] > 0

    grouped_stats = df.groupby('DTE').agg(
        returns_sum=('ROI%', 'sum'),
        max_drawdown=('DD', 'min'),
        total_trades=('DTE', 'count'),
        wins=('is_win', 'sum')
    ).reset_index()

    grouped_stats['returns_to_max_dd_ratio'] = grouped_stats['returns_sum'] / grouped_stats['max_drawdown'].abs()
    grouped_stats['win_rate'] = (grouped_stats['wins'] / grouped_stats['total_trades']) * 100

    return grouped_stats

import pandas as pd

def calculate_trade_num_stats(df):
    if 'trade_num' not in df.columns or 'ROI%' not in df.columns or 'DD' not in df.columns:
        raise ValueError("DataFrame must contain 'trade_num', 'ROI%', and 'DD' columns.")
    
    df['is_win'] = df['ROI%'] > 0

    grouped_stats = df.groupby('trade_num').agg(
        returns_sum=('ROI%', 'sum'),
        max_drawdown=('DD', 'min'),
        total_trades=('trade_num', 'count'),
        wins=('is_win', 'sum')
    ).reset_index()

    grouped_stats['returns_to_max_dd_ratio'] = grouped_stats['returns_sum'] / grouped_stats['max_drawdown'].abs()
    grouped_stats['win_rate'] = (grouped_stats['wins'] / grouped_stats['total_trades']) * 100

    return grouped_stats


In [87]:
stats_dte = calculate_dte_stats(tb)
stats_dte

,DTE,returns_sum,max_drawdown,total_trades,wins,returns_to_max_dd_ratio,win_rate
0,0,30.7697,-34.9032,161,112,0.8816,69.5652
1,1,91.8896,-35.0380,171,90,2.6226,52.6316
2,2,107.4218,-33.7671,172,80,3.1813,46.5116
3,3,88.7933,-37.6326,238,88,2.3595,36.9748
4,5,15.1737,-31.7455,24,9,0.4780,37.5000
5,6,57.0227,-38.2511,555,160,1.4907,28.8288


In [88]:
stats_opt_type = calculate_opt_type_stats(tb)
stats_opt_type

,type,returns_sum,max_drawdown,total_trades,wins,returns_to_max_dd_ratio,win_rate
0,C,122.1303,-38.2511,698,265,3.1929,37.9656
1,P,268.9405,-35.3389,623,274,7.6103,43.9807


In [89]:
stats_trade_num_ce = calculate_trade_num_stats(tb_ce)
stats_trade_num_ce

/tmp/ipykernel_43052/1347911221.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['is_win'] = df['ROI%'] > 0


,trade_num,returns_sum,max_drawdown,total_trades,wins,returns_to_max_dd_ratio,win_rate
0,1,42.3013,-31.9749,272,85,1.3230,31.2500
1,2,33.5812,-38.2511,182,70,0.8779,38.4615
2,3,31.8757,-34.9032,112,50,0.9133,44.6429
3,4,-1.6845,-37.6326,62,27,-0.0448,43.5484
4,5,12.9340,-33.9812,34,16,0.3806,47.0588
5,6,-4.8385,-34.4821,17,7,-0.1403,41.1765
6,7,-4.3389,-26.4188,10,3,-0.1642,30.0000
7,8,3.3461,-26.7497,7,5,0.1251,71.4286
8,9,8.9538,-18.0399,2,2,0.4963,100.0000


In [35]:
stats_trade_num_pe = calculate_trade_num_stats(tb_pe)
stats_trade_num_pe

/tmp/ipykernel_98042/1347911221.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['is_win'] = df['ROI%'] > 0


,trade_num,returns_sum,max_drawdown,total_trades,wins,returns_to_max_dd_ratio,win_rate
0,1,112.2430,-35.1491,280,117,3.1933,41.7857
1,2,91.3206,-35.0380,163,71,2.6063,43.5583
2,3,8.2959,-35.3389,86,37,0.2348,43.0233
3,4,30.3842,-28.8209,48,26,1.0542,54.1667
4,5,20.3025,-22.3567,22,12,0.9081,54.5455
5,6,4.1628,-21.4691,11,5,0.1939,45.4545
6,7,-2.8553,-10.0477,6,2,-0.2842,33.3333
7,8,4.1217,-7.8374,4,2,0.5259,50.0000
8,9,-0.5145,-9.4429,2,1,-0.0545,50.0000
9,10,1.4796,-10.5902,1,1,0.1397,100.0000


In [36]:
tb.tail(25)

,date,day,expiry,DTE,trade_num,strike,type,Entry Price,Entry Time,initial sl,Exit Price,Exit date,Exit Time,ROC on Entry,Opt RSI on Entry,Spot RSI on Entry,Remark,Points Captured,Slippage,Qty,PnL,ROI%,Trade Year,Trade Month,Highest High,Lowest Low,Max ROI%,Margin,DATETIME,Cumulative ROI%,Max Cumulative ROI%,DD,is_win
688,2025-01-24,Friday,2025-01-30,6,2,23300,C,92.1000,14:15:00,215.5000,0.1500,2025-01-30,15:15:00,-38.5971,0.0000,52.1919,EOD exit,91.9500,0.9225,3004.2918,273473.1760,2.7347,2025,1,107.1000,0.1000,2.7639,100.0000,2025-01-24 14:15:00,364.1605,391.0560,-26.8955,True
1306,2025-01-28,Tuesday,2025-01-30,2,2,22900,P,73.8000,13:45:00,182.0000,0.1000,2025-01-30,15:15:00,-22.8033,17.6211,17.8947,EOD exit,73.7000,0.7390,3056.7686,223024.8908,2.2302,2025,1,129.3500,0.0500,2.2544,100.0000,2025-01-28 13:45:00,366.3907,391.0560,-24.6653,True
1307,2025-01-31,Friday,2025-02-06,6,1,23300,P,211.0000,10:15:00,305.1000,0.0500,2025-02-06,15:15:00,-51.1851,0.0000,74.1067,EOD exit,210.9500,2.1105,3004.2918,627414.8069,6.2741,2025,2,222.6000,0.0500,6.3376,100.0000,2025-01-31 10:15:00,372.6649,391.0560,-18.3911,True
689,2025-02-01,Saturday,2025-02-06,5,1,23550,C,169.7500,11:45:00,295.1500,53.2000,2025-02-06,15:15:00,-30.0640,18.9699,43.9187,EOD exit,116.5500,2.2295,2972.3992,339806.1571,3.3981,2025,2,233.7500,14.0000,4.6295,100.0000,2025-02-01 11:45:00,376.0629,391.0560,-14.9931,True
690,2025-02-07,Friday,2025-02-13,6,1,23600,C,194.4500,10:45:00,234.8500,234.8500,2025-02-07,11:15:00,-33.0164,39.9527,50.6994,SL hit,-40.4000,4.2930,2966.1017,-132563.9831,-1.3256,2025,2,248.8000,192.0500,0.0712,100.0000,2025-02-07 10:45:00,374.7373,391.0560,-16.3187,False
1308,2025-02-07,Friday,2025-02-13,6,1,23550,P,91.0000,11:15:00,177.9000,177.9000,2025-02-07,13:45:00,-22.9764,20.6309,70.6187,SL hit,-86.9000,2.6890,2972.3992,-266294.2675,-2.6629,2025,2,200.0000,88.5500,0.0728,100.0000,2025-02-07 11:15:00,372.0744,391.0560,-18.9817,False
691,2025-02-07,Friday,2025-02-13,6,2,23700,C,108.6500,11:45:00,186.8000,0.0500,2025-02-13,15:15:00,-51.9876,32.5479,43.9187,EOD exit,108.6000,1.0870,2953.5865,317548.9451,3.1755,2025,2,123.3000,0.0500,3.2076,100.0000,2025-02-07 11:45:00,375.2498,391.0560,-15.8062,True
1309,2025-02-13,Thursday,2025-02-13,0,2,23000,P,20.4500,09:45:00,105.0000,0.0500,2025-02-13,15:15:00,-24.0672,0.0000,88.2979,EOD exit,20.4000,0.2050,3043.4783,61463.0435,0.6146,2025,2,31.4000,0.0500,0.6209,100.0000,2025-02-13 09:45:00,375.8645,391.0560,-15.1915,True
1310,2025-02-14,Friday,2025-02-20,6,1,23000,P,202.5500,10:15:00,209.0000,209.0000,2025-02-14,10:45:00,-7.9183,71.7878,74.1067,SL hit,-6.4500,4.1155,3043.4783,-32155.8696,-0.3216,2025,2,248.2500,192.8500,0.2952,100.0000,2025-02-14 10:15:00,375.5429,391.0560,-15.5131,False
692,2025-02-14,Friday,2025-02-20,6,1,23050,C,136.9500,10:15:00,236.0000,0.0500,2025-02-20,15:15:00,-21.4796,13.8153,78.5698,EOD exit,136.9000,1.3700,3036.8764,411587.8525,4.1159,2025,2,158.1000,0.0500,4.1575,100.0000,2025-02-14 10:15:00,379.6588,391.0560,-11.3972,True


In [37]:
tb_ce = tb[tb['type'] == 'C']
tb_pe = tb[tb['type'] == 'P']

In [41]:
tb.to_csv('kitkat_30m.csv')

In [38]:
# tb = tb.sort_values(by='DATETIME')
stats = generate_stats(tb_ce, '...')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,20.1415,107,37.3832,2.1955,-1.0101,-16.1508,1.2471,...
2020,55.7293,112,39.2857,4.0851,-1.8237,-42.7936,1.3023,...
2021,-2.6180,125,35.2000,2.1090,-1.1780,-22.0653,-0.1186,...
2022,4.1712,108,40.7407,2.5621,-1.6963,-31.0238,0.1345,...
2023,19.6201,114,36.8421,1.9054,-0.8390,-11.1234,1.7639,...
2024,6.7494,115,35.6522,2.1845,-1.1191,-23.9307,0.2820,...
2025,18.3368,17,58.8235,3.1485,-1.8783,-2.6426,6.9388,...
Overall,122.1303,698,37.9656,2.5440,-1.2749,-49.3006,2.4773,...


In [39]:
# tb = tb.sort_values(by='DATETIME')
stats = generate_stats(tb_pe, '...')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,20.4804,93,41.9355,2.2242,-1.2271,-16.4592,1.2443,...
2020,155.0707,92,52.1739,5.1053,-2.0451,-28.3847,5.4632,...
2021,33.0275,103,42.7184,3.2486,-1.8629,-24.3848,1.3544,...
2022,46.9246,116,38.7931,3.6615,-1.6597,-37.1163,1.2643,...
2023,19.1679,97,43.2990,1.8389,-1.0557,-10.1556,1.8874,...
2024,-9.4399,95,45.2632,2.0211,-1.8528,-19.2758,-0.4897,...
2025,3.7094,27,48.1481,2.2807,-1.8528,-11.3432,0.3270,...
Overall,268.9405,623,43.9807,3.0412,-1.6171,-37.1163,7.2459,...


In [40]:
tb = tb.sort_values(by='DATETIME')
stats = generate_stats(tb, '...')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,40.6219,200,39.5000,2.2096,-1.1069,-24.9886,1.6256,...
2020,210.8000,204,45.0980,4.6174,-1.9107,-34.4821,6.1133,...
2021,30.4095,228,38.5965,2.6788,-1.4666,-37.6326,0.8081,...
2022,51.0958,224,39.7321,3.1180,-1.6771,-38.2511,1.3358,...
2023,38.7879,211,39.8104,1.8722,-0.9329,-9.4427,4.1077,...
2024,-2.6905,210,40.0000,2.1008,-1.4219,-29.1357,-0.0923,...
2025,22.0462,44,52.2727,2.6580,-1.8613,-8.1611,2.7014,...
Overall,391.0708,1321,40.8024,2.7968,-1.4276,-38.2511,10.2238,...


In [26]:
tb_margin = tb[tb['Margin'] > 100]
tb_margin['ROI%'].sum(), len(tb_margin), len(tb)

(534.3461430674945, 1053, 1458)

In [25]:
# tb.to_csv('EMAR_5m_50EMA.csv', index=False)

In [27]:
# tb.tail(50)

In [37]:
stats = generate_stats(tb, '...')
for x, y in stats.items():
    z = pd.DataFrame(y)
    break

z

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2019,113.1501,230,34.3478,5.0277,-1.8810,-22.5788,5.0113,...
2020,134.9478,256,34.3750,5.8342,-2.2527,-35.2967,3.8232,...
2021,97.6609,233,36.0515,4.9230,-2.1199,-21.8114,4.4775,...
2022,103.6360,248,31.8548,6.3140,-2.3383,-36.9616,2.8039,...
2023,119.8259,217,35.9447,4.9810,-1.9330,-14.0168,8.5487,...
2024,45.9805,231,32.0346,4.9413,-2.0362,-31.7348,1.4489,...
2025,46.0862,44,38.6364,5.9487,-2.0386,-8.8266,5.2213,...
Overall,661.2874,1459,34.2015,5.3672,-2.1010,-36.9616,17.8912,...


# Simulation

In [22]:
TF_ = ['3m', '15m', '30m', '10m', '5m']
# EMA_ = [18]
# RSI_ = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]
# MA_Range = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]
# RSI_THRESHOLD_ = [30, 40, 50, 60, 70, 80]

start_dt = dt.date(2019, 1, 1)
end_dt = dt.date(2025, 3, 31)

for tf in TF_:
    for roc in range(5, 51, 5):
        variation = f'{tf} , {roc}'
        print(variation)
        if tf == '10m':
            offset = '5m'
        elif tf >= '20m':
            offset = '15m'
        else:
            offset = '0m'
        bnfx = resample(bnf, tf, offset)
        data = bnfx.to_pandas()
        tb_ce = await ce_trade(data, tf, offset, roc, 10, start_dt, end_dt)
        tb_pe = await pe_trade(data, tf, offset, roc, 10, start_dt, end_dt)
        tb = pd.concat([tb_ce, tb_pe], ignore_index=True)

        if len(tb) > 0:
            tb = tb.sort_values(by="date")
            tb['DATETIME'] = pd.to_datetime(tb['date'].astype(str) + ' ' + tb['Entry Time'].astype(str))
            tb = tb.sort_values(by='DATETIME')
            stats = generate_stats(tb, variation)
            for x, y in stats.items():
                z = pd.DataFrame(y)
                print(z.to_string())

                        

3m , 5
        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio Variation
2019      72.8247          345  26.3768                3.1549             -0.8436     -21.4955       3.3879    3m , 5
2020      86.3949          437  24.2563                6.0031             -1.6614     -94.3337       0.9158    3m , 5
2021      52.3410          429  24.2424                3.7245             -1.0308     -33.1922       1.5769    3m , 5
2022      54.4259          448  22.7679                3.9820             -1.0166     -24.5609       2.2160    3m , 5
2023      29.4792          419  23.3890                2.3618             -0.6292     -15.8097       1.8646    3m , 5
2024       8.8148          434  22.8111                3.3396             -0.9606     -44.3814       0.1986    3m , 5
2025      20.4118          102  25.4902                3.5587             -0.9489      -9.5835       2.1299    3m , 5
Overall  324.6923         2614  23.9480          

CancelledError: 